Tutorial:
=========
Extracting a time-ordered list of processes that occurred during deposition from an AJA logfile. In order to run this, you'll have to have the required directories of AJA logfiles, recipe files, and job files.

In [1]:
#Add the AJA repository parent directory to $PATH
import sys
sys.path.append('path_to_parentdir')

import glob
import pprint

from AJA_tools import recipe_parser as aja

Specify file locations
---------------------
The easiest thing is to put all the recipe files (`.rcp`) in one folder and all the job files (`.ajp`) in another. I did this by opening a terminal and typing::

    mkdir recipes
    mv *.rcp recipes
    mkdir jobs
    mv *.ajp jobs

In [2]:
recipe_files_dir = './recipes/'
job_files_dir = './jobs/'

Parse files and generate database of jobs
---------------------------------------
This function takes in a folder of `.ajp` files and builds a dictionary of jobs that matches job names to job file locations.

In [3]:
#Build up the dictionary of jobs
jobs_dict = aja.build_jobs_dict(job_files_dir)

#Print the first couple of job names
print("The first five jobs are:")
pprint.pprint(list(jobs_dict.keys())[0:5])

The first five jobs are:
['Ci Ti deposition 3mins',
 'CLEAN_RF25W_2h_wafer3',
 'CP 30min Ti deposition_NO RF_wait_1hr',
 'CP 3X 30min Ti deposition_NO RF_wait_2hr_in_between',
 'CP Heat 4hr 250C']


Get the path to the right job file
--------------------------------
`get_job` takes in a logfile path and a dictionary of jobs (or path to a jobs folder) and returns both the job name and the job file.

In [4]:
#Specify the logfile of interest
log_file_path = './datalog/SL SiO2 Gun4 1hr 200C 274 026 200W_192015_ 51401 PM.dlg'

job_name, job_file_path = aja.get_job(log_file_path, jobs_dict)
print("Job name is: "+job_name)
print("Job file is: "+job_file_path)

Job name is: SL SiO2 Gun4 1hr 200C 274 026 200W
Job file is: ./jobs/SL SiO2 Gun4 1hr 200C 274 026 200W.ajp


Extract the recipe from the job file
----------------------------------
Since the job files all have a specific format, passing in the recipe files directory path is optional. However, if that path is passed in, the function will check if each recipe step exists as a recipe file in the directory.

In [5]:
recipe_dict = aja.get_recipe(job_file_path, recipe_files_dir)
pprint.pprint(recipe_dict)

{'info': 'raw_recipe format is: (string index, string, recipe exists?)',
 'raw_job': '\x00\x00\x00\x08\x00\x00\x00!SiO2 SL  PROCESS-- HEAT 150C '
            '5MIN\x00\x00\x00!SiO2 SL  PROCESS-- HEAT 180C '
            '5MIN\x00\x00\x00!SiO2 SL  PROCESS-- HEAT 200C '
            '5MIN\x00\x00\x00"SiO2 SL PROCESS-- STRIKE GUN4 '
            '200C\x00\x00\x00.SiO2 SL PROCESS-- CLEAN PRESPUT 3min 200C '
            'GUN4\x00\x00\x00#SiO2 SL PROCESS-- DEP 1hr 200C '
            'GUN4\x00\x00\x00\x1e'
            'SiO2 SL PROCESS-- TURNOFF GUN4\x00\x00\x00%SiO2 SL PROCESS-- '
            'TURNOFF GAS AND ROT',
 'raw_recipe': [(0, '\x00\x00\x00\x08\x00\x00\x00!', None),
                (8, 'SiO2 SL  PROCESS-- HEAT 150C 5MIN', True),
                (41, '\x00\x00\x00!', None),
                (45, 'SiO2 SL  PROCESS-- HEAT 180C 5MIN', True),
                (78, '\x00\x00\x00!', None),
                (82, 'SiO2 SL  PROCESS-- HEAT 200C 5MIN', True),
                (115, '\x00\x00\x00"', None

Check for missing job files
-------------------------
Now that we've gone through all this, we may as well check to see if any of the logfiles don't have jobs associated with them.

In [10]:
logfiles = glob.glob('./datalog/*.dlg')
for log in logfiles:
    job_name, job_file = aja.get_job(log, jobs_dict)
    if job_file is None:
        print(job_name)

CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
rg_lu_3_4_nb_40mins_30sec_1
rg_lu_3_4_nb_40mins_30sec_1
rg_lu_3_4_nb_40mins_30sec_1
rg_lu_3_4_nb_40mins_30sec_1
rg_lu_3_4_nb_40mins_30sec_1



/Users/fcarter/Documents/Code/Python/AJA_tools/recipe_parser.py:102: UserWarning: No matching job found!
  warnings.warn("No matching job found!", UserWarning)
